# Non-cooperative Evolutionary Game
Biological populations evolve under pressure from a combination of factors: random genetic mutations, unpredictable environmental disturbance and interactions between each other. Natural selection can be abstracted into the idea of there being strategies 'chosen' by evolutionary agents. These strategies consist of the selection of certain genetic mutations that arise in the population over time and they have the underlying goal of maximizing their own fitness in preserving the proportion of stable phenotypes in the population.

In the paper *Stochastic noncooperative and cooperative evolutionary game strategies of a population of biological networks under natural selection, Biosystems* by Chen et al., this is modelled as both non-cooperative and co-operative evolutionary games. In this notebook, we focus on the non-cooperative case.

We first look at modeling the rate of change of different species in a population which will then be used to demonstrate how effective certain strategies are, meaning how effectively they can counter disturbances and maintain a stable population state with minimal evolutionary effort.

## Nonlinear Evolutionary Biological Network
The mathematical model of a Nonlinear Evolutionary Biological Network takes into account the effects of the following on a biological network:
- nonlinear interactions between species
- genetic mutations that have occurred randomly up to a certain time
- random environmental disturbances

and will be used to demonstrate how robust chosen strategies are for how the population state evolves. Strategies of the population are constructed based on choosing genetic mutations that have occurred in the population so far. The effect of these strategies can be viewed by looking at how a population state $\dot{x}(t)$ changes through the following equation:

$$\dot{x}(t) = f(x) + \sum_{i=1}^m u_i(t) + \sum_{k=1}^{N(t)} f_k(t)p(t - t_k) + Bv(t)$$
where:
- $t$ is the current time that we are looking at a biological system
- $x(t) = [x_1(t) ... x_i(t) ... x_n(t)]^T$ is the current population state for n species at time $t$. $x_i(t)$ represents the proportion of species $i$ at time $t$.
- $f(x)$ is the nonlinear interaction vector among n different species in the biological network
- $u_i(t)$ is payoff or fitness contribution of a strategy $i$ (we can also term this player $i$) in the biological network
    - There are $m$ players which is why we sum together all of their payoffs $\sum_{i=1}^m u_i(t)$
- $N(t)$ is the number of genetic mutation events that have occurred up until the current time $t$
    - This is calculated using a Poisson process and the $k$th mutation event occurs at time $t_k$
- $p(t - t_k)$ is the unit step function where $p(t - t_k) = 1$ if $t_k <= t$ (meaning the $k$th mutation has occurred by time $t$) and 0 otherwise
    - Since $\sum_{k=1}^{N(t)} f_k(t)p(t - t_k)$ goes through the $N(t)$ mutation events that have occurred until time $t$, $p(t - t_k)$ will always equal 1 and this can be simplified to $\sum_{k=1}^{N(t)} f_k(t)$.
- $v(t)$ represents the random environmental disturbances
- $B$ is the coupling matrix from environmental disturbances to the biological network
    - Hence, $Bv(t)$ calculates the effect of the environmental disturbances on the biological network

## Linearized Evolutionary Biological Network
For simplicity, we will look at the linearized version of this network where $f(x)$ is replaced with the matrix multiplication $A\tilde{x}(t)$. $A$ is considered to be the derivative of $f(x)$ at an equilibrium point $x_e(t)$ where the phenotypes of the species are stable. We use $\tilde{x}(t) = x(t) - x_e(t)$ to be the deviation of the population state from the equilibrium point:
$$\dot{\tilde{x}}(t) = A\tilde{x}(t) + \sum_{i=1}^m u_i(t) + \sum_{k=1}^{N(t)} A_k\tilde{x}(t) + Bv(t)$$

We also define the strategy payoff $u_i(t)$ for a player $i$ to be constructed as if the player 'chose' from certain genetic mutations that have occurred: $u_i(t) = \sum_{k=1}^{N_i(t)} A_kp(t - t_k)\tilde{x}(t)$ where $N_i(t)$ are mutations that player $i$ chose.

Since each player chooses their own mutations, we let $N_r(t)$ be the set of mutations that were not chosen and $\sum_{k=1}^{N_r(t)} A_kp(t - t_k)\tilde{x}(t)$ be the sum of payoffs of mutations that were not chosen by any strategy. These are then considered to be neutral genetic variations that are just noise.

Because our aim is to model this as an optimization problem for each strategy $i$, then if we look at the evolutionary game in the perspective of a player/strategy $i$, we know that all other players' strategies are unable to be manipulated by this player. Furthermore, the environmental disturbances $v(t)$ are also unable to be changed by the player. We can then term the environment as another adversary and bundle together all the adversaries as follows:
$$\bar{v}_i(t) = [v(t), u_1(t), ..., u_{i-1}(t), u_{i+1}(t), ..., u_m(t)]^T$$
and let
$$\bar{B}_i = [B, I, ..., I]$$
So we have a new version of the equation:
$$\dot{\tilde{x}}(t) = A\tilde{x}(t) + \sum_{k=1}^{N_r(t)} A_kp(t - t_k)\tilde{x}(t) + u_i(t) + \bar{B}_i\bar{v}_i(t) \qquad (1)$$

## Finding robust strategies
We now look at the core of the problem: finding robust strategies within the non-cooperative game that make an effort to preserve a stable phenotype. We can look at this through the lens of the evolvability of each strategy/player $i$ which measures how well player $i$ can adapt to disturbance relative to how much disturbance existed. We term this $e_i^*$ and define it as:
$$ e_i^* = \min_{u_i(t)} \max_{\bar{v}_i(t)} \frac{E[\int_0^{t_p} (\tilde{x}^T(t)Q_i\tilde{x}(t) + u_i^T(t)R_iu_i(t))dt]}{E[\tilde{x}^T(0)\tilde{x}(0) + \int_0^{t_p} \bar{v}_i^T(t) \bar{v}_i(t) dt]}$$

Overall, the problem description shows we would like to pick the best strategy $u_i(t)$ that minimizes the following two objectives:
- the deviation from the equilibrium point ($\tilde{x}(t_p) = x(t_p) - x_e(t_p)$) up until the present time $t_p$: $\int_0^{t_p} (\tilde{x}^T(t)Q_i\tilde{x}(t)$
    - In other words, this means to preserve the current phenotypic states of the population in the face of both neutral genetic mutations as well as the worst-case effects of adversarial strategies
    - $Q_i$ is the weighting matrix that describes the penalty of deviating from the equilibrium point. This means how important it is for a species to stick to its stable phenotype. Large values in $Q_i$ would mean that the species cares strongly about minimizing deviation whereas small values mean it matters less.  
- the effort required to do this (a.k.a the strategy $u_i(t)$) up until the present time $t_p$: $\int_0^{t_p} u_i^T(t)R_iu_i(t))dt$
    - $R_i$ is the weighting matrix that describes the cost of using available genetic mutations which is why it is multiplied with the strategy that is composed of the selected mutations. Large values in $R_i$ would mean that it costs a lot to mutate, so the strategy should be conservative. Small values in $R_i$ mean it selects mutations more agressively.

At the same time, the adversaries $\max_{\bar{v}_i(t)}$ work to maximize the deviation from the equilibrium point. This is why it is a non-cooperative game.

The reason for the denominator is to normalize the fitness measurement by taking into account how unfit the initial conditions were ($\tilde{x}^T(0)\tilde{x}(0)$) and how strong the adversary disturbances were ($\int_0^{t_p} \bar{v}_i^T(t) \bar{v}_i(t) dt$). So, a large $e_i^*$ means the player is highly evolvable and a small $e_i^*$ means it cannot efficiently counter disturbances.

## Reformulating the Optimization Problem
Since it is difficult to solve the above optimization problem for m strategies, we can solve a suboptimal multi-objective problem that minimizes the upper bounds $e_i$ which approximates the true evolvabilities $e_i^*$ for $i = 1, ..., m$:
$$ \min (e_1, ..., e_m)$$
$$\text{subject to } \min_{u_i(t)} \max_{\bar{v}_i(t)} \frac{E[\int_0^{t_p} (\tilde{x}^T(t)Q_i\tilde{x}(t) + u_i^T(t)R_iu_i(t))dt]}{E[\tilde{x}^T(0)\tilde{x}(0) + \int_0^{t_p} \bar{v}_i^T(t) \bar{v}_i(t) dt]} \leq e_i, \forall i = 1, ..., m$$

With further derivations that you can read in the paper, we can get that the constrained stochastic Nash games above can be solved by the following n-person noncooperative evolutionary game strategies $u_i^*(t)$ and the worst-case combined competitive strategies and environmental disturbances $\bar{v}_i^*(t)$ for $i = 1, ..., m$:
$$ u_i^*(t) = \sum_{k=1}^{N_i(t)} A_{k}p(t - t_k)\tilde{x}(t) = -R_i^{-1}P\tilde{x}(t) \qquad (2)$$ and
$$\bar{v}_i^*(t) = \frac{1}{e_i}\bar{B}_i^TP\tilde{x}(t) \qquad (3)$$
where the positive definite matrix $P$ is the solutions to the following Ricatti-like inequalities:
$$A^{T}P + PA - PR_i^{-1}P + \frac{1}{e_i} P \bar{B}_i \bar{B}_i^{T} P + Q_i 
+ \sum_{k=1}^{N_{rp}} \left( A_k^{T} P + P A_k + A_k^{T} P A_k \right) \le 0 , 
\qquad (*) \qquad \text{for } i = 1,2,\ldots,m$$
$$\text{with } 0 < P \le e_i I,\quad \text{for } i = 1,2,\ldots,m$$

where $N_{rp} = Nr(t_p)$ represents the total number of remaining neutral genetic variations to the present time $t_p$ after selection by $m$ players.
- The term $A^{T}P + PA - PR_i^{-1}P$ models the effect of non-cooperative evolutionary game strategies
- The term $\frac{1}{e_i} P \bar{B}_i \bar{B}_i^{T} P + Q_i$ models the effect of other strategies and environmental disturbance
- The term $\sum_{k=1}^{N_{rp}} \left( A_k^{T} P + P A_k + A_k^{T} P A_k \right)$ models the effect of random genetic variations that are not selected by the m evolutionary strategies of m players, i.e., of neutral genetic variations.

Since the previous constraints are Ricatti-like inequalities that cannot be easily solved, we can set $W = P^{-1}$ on both sides of the inequalities and use the Schur complement method to get the following linear matrix inequalities-constrained multi-objective problem.

$$
\bigl(e_1^{*}\ \cdots\ e_i^{*}\ \cdots\ e_m^{*}\bigr)
= \min_{W \succ 0} \ \bigl(e_1\ \cdots\ e_i\ \cdots\ e_m\bigr)
$$

$$
\text{subject to }
$$
$$
\begin{bmatrix}
WA^{\top} + AW - R_i^{-1} + \dfrac{1}{e_i}\,\bar{B}_i \bar{B}_i^{\top}
+ \displaystyle\sum_{k=1}^{N_{rp}} \bigl(WA_k^{\top} + A_k W\bigr)
& W & (A_1 W)^{\top} & \cdots & (A_{N_{rp}} W)^{\top} \\[6pt]
W & -Q_i^{-1} & 0 & \cdots & 0 \\[4pt]
A_1 W & 0 & -W & \cdots & 0 \\[2pt]
\vdots & \vdots & \vdots & \ddots & \vdots \\[2pt]
A_{N_{rp}} W & 0 & 0 & \cdots & -W
\end{bmatrix}
\preceq 0,
$$

$$
\begin{bmatrix}
W & I \\
I & e_i I
\end{bmatrix}
\succeq 0 ,
\text{for } i=1,2,\ldots,m,
$$

Since this is a multi-objective problem, an optimal global solution may not exist so we must solve for a set of Pareto optimal solutions. These solutions are a good compromise such that no one player can be made better without making another worse off.

In order to actually calculate optimal $u_i^*(t)$ and $\bar{v}_i^*(t)$, the paper by Chen et al. describes an algorithm in Section 2.1 wich we will implement. The general outline is:
1. Use an evolutionary algorithm to solve the optimization problem and generate a Pareto front of solutions $(e_1 ...e_m)$ and $W$ that satisfy the above constraints.
2. Since $W = P^{-1}$, we compute $P$ and use it to compute $u_i^*(t)$ and $\bar{v}_i^*(t)$
3. We then substitute equation (2) into the equation (1) for $\dot{\tilde{x}}(t)$ to get the rate of change of the population state from the optimal strategies as
$$
 \dot{\tilde{x}}(t) = \left( A - \sum_{i=1}^{m} R_i^{-1} P^* \right) \tilde{x}(t)
 + \sum_{k=1}^{N_r(t)} A_k p(t - t_k) \tilde{x}(t) + Bv(t) $$
Note that $\left( A - \sum_{i=1}^{m} R_i^{-1} P^* \right)$ shifts the eigenvalues left, meaning their real part becomes more negative which means that it becomes more robust to noise and returns to equilibrium more quickly.
4. We calculate the next $\tilde{x}(t+\Delta{t}) = \tilde{x}(t) + \Delta{t}\dot{\tilde{x}}(t)$ 
5. We plot the $\tilde{x}(t)$ over a sequences of times $t$ to view the deviation of x from the equilibrium point.